# Calculating Charity Scores from relevant charity data on charitydata.ca (using dataframe from Data_Collection.ipyntb)

In [1]:
#importing required libraries and setting the driver path
import pandas as pd
import numpy as np
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

driver_path = "C:/Users/16475/Documents/GitHub/CharityScore.ca/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)

In [2]:
# import the url data from the dataframe
df = pd.read_csv("expense data.csv")

# remove duplicate index column
df = pd.read_csv("expense data.csv").drop(['Unnamed: 0'],axis=1)

print(df.head(1))

                                                 URL       Name BRN  \
0  https://www.charitydata.ca/charity/full-name/0...  full name   1   

  Expense Table Expense_Table  
0           NaN           NaN  


In [3]:
# find expense tables on charitydata.ca and add them to the dataframe as string representations of dictionaries

for i in range(1, 1):
    driver.get(df.loc[i,"URL"])
    
    # check if url page found, else skip
    try:
        url_check = driver.find_element_by_xpath("/html/body/main/div/section[2]/div/div[2]/p").text
    except NoSuchElementException:
        url_check = ""
    
    if url_check == "The requested page could not be found.":
        continue
    
    # open expense table
    try:
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.LINK_TEXT, 'Expenses'))).click()
        expense_table = driver.find_element_by_xpath('//html/body/main/div/section[3]/div/div[2]/div[4]/div[2]/table[1]')
    except TimeoutException:
        try:
            driver.get(df.loc[i,"URL"])
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.LINK_TEXT, 'Expenses'))).click()
            expense_table = driver.find_element_by_xpath('//html/body/main/div/section[3]/div/div[2]/div[4]/div[2]/table[1]')
        except:
            continue
    
    # create dictionary representation of expense table
    expense_dict = {}
    
    expense_rows = expense_table.find_elements(By.TAG_NAME, "tr")
    header_row = expense_rows.pop(0)
    
    headers = header_row.find_elements(By.TAG_NAME, "th")
    header_key = headers.pop(0).text
    headers = list(map(lambda x: int(x.text), headers))
    expense_dict[header_key] = headers 
    
    for row in expense_rows:
        row_key = row.find_element(By.TAG_NAME, "th").text
        row_series = row.find_elements(By.TAG_NAME, "td")
        num_list = []
        
        for num in row_series:
            if num.text[0] != "(":
                num_list +=[int(num.text.replace(",",""))]
            else:
                num_list += [int(num.text[1:-1].replace(",",""))]
                
        expense_dict[row_key] = num_list
        
    df.loc[df.index[i], 'Expense_Table'] = str(expense_dict)
    df.to_csv("expense data.csv")
    
# quit driver and export to csv  
driver.quit()
df.to_csv("expense data.csv")
    
    
    
    

KeyError: 84079

In [27]:
# Adding columns for averages and percentages of expenditures per category

import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt

# import the url data from the dataframe
df = pd.read_csv("expense data.csv")

# remove duplicate index column
df = pd.read_csv("expense data.csv").drop(['Unnamed: 0'],axis=1)

# create list of common keys to access by index rather than name
ck = list(ast.literal_eval(df.loc[1,"Expense Table"]).keys())

# create data labels from common keys


# for each line in the dataframe, find the expense dictionary and create arrays for each bucket 
for i in range(1,len(df)):
    if df.loc[i, "Expense Table"] != "":
        try:
            expense_dict = ast.literal_eval(df.loc[i, "Expense Table"])
        except ValueError:
            expense_dict = None
    
    if expense_dict != None:
        reporting_years = expense_dict[ck[0]]
        total_exp = expense_dict[ck[7]]
        charitable_exp = expense_dict[ck[1]]
        mgmt_exp = expense_dict[ck[2]]
        fundraising_exp = expense_dict[ck[3]]
        pol_exp = expense_dict[ck[4]]
        other_exp = expense_dict[ck[5]]
        gift_exp = expense_dict[ck[6]]
        
        expense_matrix = np.matrix([reporting_years, total_exp, charitable_exp, mgmt_exp, fundraising_exp, pol_exp, gift_exp, other_exp])
        
        # create a new columns with values from most recent year
        df.loc[i, 'Reporting Year'] = expense_matrix[:,0][0].item()
        df.loc[i, 'Total Expenditure'] = expense_matrix[:,0][1].item()
        df.loc[i, 'Total Expenditure on Charitable Activities'] = expense_matrix[:,0][2].item()
        df.loc[i, 'Total Expenditure on Management and Administration'] = expense_matrix[:,0][3].item()
        df.loc[i, 'Total Expenditures on Fundraising'] = expense_matrix[:,0][4].item()
        df.loc[i, 'Total Expenditures on Political Activities'] = expense_matrix[:,0][5].item()
        df.loc[i, 'Total Other Expenditures '] = expense_matrix[:,0][6].item()
        df.loc[i, 'Total Amount of Gifts Made to All Qualified Donees'] = expense_matrix[:,0][7].item()
        
        df.to_csv("expense data calcs.csv")
        print(expense_matrix[:,0])

[[ 2020]
 [77309]
 [75367]
 [ 1895]
 [    0]
 [    0]
 [    0]
 [    0]]
[[  2020]
 [384979]
 [341365]
 [ 43614]
 [     0]
 [     0]
 [     0]
 [     0]]
[[ 2019]
 [25277]
 [23154]
 [ 2123]
 [    0]
 [    0]
 [    0]
 [    0]]
[[  2020]
 [235640]
 [149745]
 [ 80493]
 [  5402]
 [     0]
 [     0]
 [     0]]
[[ 2019]
 [16380]
 [ 4752]
 [ 1280]
 [    0]
 [    0]
 [    0]
 [    0]]
[[2019]
 [  70]
 [   0]
 [   0]
 [   0]
 [   0]
 [   0]
 [   0]]
[[ 2019]
 [37874]
 [36299]
 [ 1575]
 [    0]
 [    0]
 [    0]
 [    0]]
[[2020]
 [1555]
 [   0]
 [   0]
 [   0]
 [   0]
 [   0]
 [   0]]
[[  2019]
 [410985]
 [349014]
 [ 57622]
 [     0]
 [     0]
 [     0]
 [  4349]]
[[2020]
 [  35]
 [1500]
 [  35]
 [   0]
 [   0]
 [   0]
 [   0]]
[[  2019]
 [468200]
 [362522]
 [105678]
 [     0]
 [     0]
 [     0]
 [     0]]
[[2020]
 [  42]
 [   0]
 [   0]
 [   0]
 [   0]
 [   0]
 [   0]]
[[ 2019]
 [39310]
 [35358]
 [ 3952]
 [    0]
 [    0]
 [    0]
 [    0]]
[[   2019]
 [5755050]
 [1703409]
 [ 311641]
 [     

[[2020]
 [1181]
 [   0]
 [   0]
 [   0]
 [   0]
 [   0]
 [   0]]
[[2020]
 [5964]
 [   0]
 [   0]
 [   0]
 [   0]
 [   0]
 [   0]]
[[ 2019]
 [12580]
 [11767]
 [    0]
 [    0]
 [    0]
 [    0]
 [  813]]
[[  2019]
 [439315]
 [439153]
 [     0]
 [     0]
 [     0]
 [     0]
 [     0]]
[[ 2019]
 [13794]
 [    0]
 [   65]
 [    0]
 [    0]
 [13729]
 [    0]]
[[ 2019]
 [18660]
 [  133]
 [18527]
 [    0]
 [    0]
 [    0]
 [    0]]
[[2020]
 [   0]
 [   0]
 [   0]
 [   0]
 [   0]
 [   0]
 [   0]]
[[   2018]
 [1003187]
 [ 851018]
 [ 110805]
 [   2973]
 [      0]
 [      0]
 [  38391]]
[[  2019]
 [176741]
 [176741]
 [     0]
 [     0]
 [     0]
 [     0]
 [     0]]
[[2019]
 [1601]
 [1547]
 [   0]
 [   0]
 [   0]
 [   0]
 [   0]]
[[ 2020]
 [75478]
 [55002]
 [18982]
 [    0]
 [    0]
 [    0]
 [    0]]
[[2019]
 [ 224]
 [   0]
 [ 224]
 [   0]
 [   0]
 [   0]
 [   0]]
[[ 2020]
 [15980]
 [15923]
 [   57]
 [    0]
 [    0]
 [    0]
 [    0]]
[[ 2019]
 [66920]
 [56360]
 [ 2075]
 [ 8485]
 [    0]
 [   